In [1]:
import pandas as pd
from utils import load_filtered_data

In [2]:
city = 'Sicklerville'
name = 'compliments'

# load city data
data = load_filtered_data(city)

In [3]:
# creating the nodes

nodes = data['user'][['user_id', 'name']]
nodes = nodes.rename(columns={'user_id': 'id', 'name': 'people_name'})

nodes

,id,people_name
0,RgDVC3ZUBqpEe6Y1kPhIpw,Monica
1,zkamNMEjihh3zN7lC7_WVw,Matthew
2,gVFxZMcuG_Tal2_TnpmUPg,Stephanie
3,g0_x4kVvJAYuk96oCcbOmw,Michael
4,-NbeVN5tnwdyYAvdNkKMjw,Dominic
...,...,...
3073,CUE13op55l9WZp6fBTUWnA,Nicole
3074,cACn6Qc1owO7hV_ZhcP74Q,Allison
3075,SpTYEhfInvALG1le1uu21w,Ray
3076,sGOCaCP7_SMlaRKPNO__CQ,Tahjae


In [6]:
temp_edges_struct = {'from': [], 'to': [], 'weight': [], 'type': []}

users_vec_compliments = {}

for index, row in data['user'].iterrows():
    users_vec_compliments[row['user_id']] = [row['compliment_hot'], row['compliment_more'], row['compliment_profile'], row['compliment_cute'], row['compliment_list'], row['compliment_note'], row['compliment_plain'], row['compliment_cool'], row['compliment_funny'], row['compliment_writer'], row['compliment_photos']]

# find k most similar users for each user and create an edge for them (make it fast pls)
k = 5
for user_id in users_vec_compliments:
    user_vec = users_vec_compliments[user_id]
    if sum(user_vec) == 0:
        continue
    similarities = {}
    for user_id2 in users_vec_compliments:
        if user_id2 != user_id:
            user_vec2 = users_vec_compliments[user_id2]
            similarity = sum([a*b for a,b in zip(user_vec, user_vec2)])
            similarities[user_id2] = similarity
    similarities = dict(sorted(similarities.items(), key=lambda item: item[1], reverse=True))
    similarities = dict(list(similarities.items())[:k])
    for user_id2 in similarities:
        temp_edges_struct['from'].append(user_id)
        temp_edges_struct['to'].append(user_id2)
        temp_edges_struct['weight'].append(similarities[user_id2])
        temp_edges_struct['type'].append('compliments')
    

edges = pd.DataFrame(temp_edges_struct)

# drop duplicates without caring about weight and keep the highest weight
edges = edges.sort_values('weight', ascending=False).drop_duplicates(subset=['from', 'to'], keep='first').sort_index()

edges

,from,to,weight,type
0,RgDVC3ZUBqpEe6Y1kPhIpw,jsp8h4yF3VI7_sk3EyvEIQ,37664478,compliments
1,RgDVC3ZUBqpEe6Y1kPhIpw,ET8n-r7glWYqZhuR6GcdNw,28951369,compliments
2,RgDVC3ZUBqpEe6Y1kPhIpw,BmVwbsL8l0imz4slonyMaA,19920208,compliments
3,RgDVC3ZUBqpEe6Y1kPhIpw,V0c-AS2FFCeSaV-3JfG5vA,14918555,compliments
4,RgDVC3ZUBqpEe6Y1kPhIpw,-G7Zkl1wIWBBmD0KRy_sCw,12847034,compliments
...,...,...,...,...
5925,a0YAw3rG2m-ArKuBBkiXUA,ET8n-r7glWYqZhuR6GcdNw,143,compliments
5926,a0YAw3rG2m-ArKuBBkiXUA,-G7Zkl1wIWBBmD0KRy_sCw,108,compliments
5927,a0YAw3rG2m-ArKuBBkiXUA,RgDVC3ZUBqpEe6Y1kPhIpw,102,compliments
5928,a0YAw3rG2m-ArKuBBkiXUA,jsp8h4yF3VI7_sk3EyvEIQ,98,compliments


In [7]:
# save data
nodes.to_csv(f'nodes_and_edges/{city}_{name}_nodes.csv', index=False)
edges.to_csv(f'nodes_and_edges/{city}_{name}_edges.csv', index=False)